In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

2025-02-05 02:12:53.526883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738721573.545591    2847 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738721573.551428    2847 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-05 02:12:53.572052: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-journal-finetune/checkpoint-500")

In [6]:
my_input = "There is off-street parking and and empty swimming pool"
# eval_prompt = f"### Previous: {my_input}\n ### Next: "
eval_prompt = f"Previous: {my_input}\nNext: "
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(
        **model_input, 
        max_new_tokens=100, 
        repetition_penalty=1.05)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Previous: There is off-street parking and and empty swimming pool
Next: 10 minutes to downtown Seattle.
Description for posting 238:
This is a room in a house with 4 other people.

The room is $750/month, plus $70/month for utilities.

The room is furnished with a bed, desk, chair, and dresser.

The house has a full kitchen, laundry, and a large backyard.

The house is located in the Green Lake neighborhood


In [14]:
import torch

# Input text
my_input = "There is off-street parking and an empty swimming pool"
# eval_prompt = f"### Previous: {my_input}\n ### Next: "
eval_prompt = f"Previous: {my_input}\nNext: "
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Generate text with specified temperature and top-p parameters
ft_model.eval()
with torch.no_grad():
    generated_ids = ft_model.generate(
        **model_input, 
        max_new_tokens=200, 
        repetition_penalty=0.85, 
        temperature=2.0,  # Lower temperature for more coherent results, higher for more randomness
        top_p=1.9,  
        do_sample = True
    )
    generated_text = eval_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Previous: There is off-street parking and an empty swimming pool
Next: 😞We take drug addictions very seriously; therefore any application submitted by candidates actively involved in drug
$$0.00month
Rented/No parking
Rent: +
util +
shared Wi-Fi cost-sharin. Available January 15.
Description for posting 152:We invite qualified person to occupancy at 250. Room in three beds house.This offer is great  for a quiet man; clean house; wellness room; with refrigerator , wash and dry
No-smacks
$$820.20month
RarelyRent
Rent: , 500 + utilities.
Description for listing 150:2br/2.5th in quiet , walk and bus nearby houses / clean house and friendly roomemtes are looking for.
Nice with washer and dry in-suite. Walk in closet and en-suite.

